# Model Evaluation (成功率計算と評価)
本ノートブックでは、前処理済みのテキストデータをBERTに入力し、Mean Poolingによる文ベクトル（Sentence Embeddings）の抽出を行います。その後、コサイン類似度を用いたk-NN的なアプローチでリアルタイムリスク判定シミュレーションを実行し、その精度を評価・可視化します。

## 1. Environment Setup & Model Loading
推論に必要なライブラリをインポートし、特徴量抽出用の事前学習済みBERTモデルをデバイス（GPU/CPU）にロードします。

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import BertTokenizer, BertModel
import random
import matplotlib.pyplot as plt

# 事前学習済みモデルの指定 (bert-base-uncased)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512)
model = BertModel.from_pretrained('bert-base-uncased')

# 計算リソースの最適化 (GPUが利用可能な場合はCUDAを使用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 2. Vector Extraction (Mean Pooling)
BERTの最終層から出力されるトークン単位の埋め込み（Token Embeddings）を、アテンションマスクを考慮しながら平均化（Mean Pooling）し、文全体の分散表現（Sentence Vectors）を抽出する関数を定義します。

In [ ]:
def get_vectors(file):
  """
  前処理済みCSVを読み込み、BERT + Mean Poolingを用いて文ベクトルを抽出する。
  """
  df_workbook = pd.read_csv(file + '.csv')
  df_workbook['cleaned_text'] = df_workbook['cleaned_text'].astype(str)
  vectors = []

  # メモリ効率のためバッチ処理を実施
  batch_size = 64
  model.eval()

  with torch.no_grad():
    for i in tqdm(range(0, len(df_workbook), batch_size)):
      batch_texts = df_workbook['cleaned_text'].iloc[i : i + batch_size].tolist()
      # テキストのトークナイズとパディング
      tmp = tokenizer.batch_encode_plus(batch_texts, truncation=True, padding=True, max_length=512, return_tensors='pt').to(device)

      outputs = model(**tmp)

      # トークンレベルの埋め込みを取得
      token_embeddings = outputs[0]

      # Attention Maskを展開し、パディング部分をゼロにする
      input_mask_expanded = tmp['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
      
      # Mean Pooling: 有効なトークンのベクトル和をトークン数で割り、文ベクトルを算出
      batch_vectors = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

      # GPUからCPUへテンソルを移動し、リストに格納
      vectors.extend(batch_vectors.cpu().numpy())

  return vectors

## 3. Evaluation Metrics Tracker
判定結果（正解/不正解）と、予測における確信度（1位と2位の類似度スコア差）および最大類似度を記録するためのトラッキング関数を定義します。

In [ ]:
def check_classification(answer_label, prediction_labels, dif_similarity_scores, most_similarity_scores):
  """
  分類結果とスコアを評価し、各種メトリクスをトラッキングする。
  """
  true_num, false_num = 0, 0
  true_dif_sims, false_dif_sims = [], []
  true_most_sims, false_most_sims = [], []

  for k in range(len(prediction_labels)):
    if answer_label == prediction_labels[k]: # 正解（True Positive / True Negative）
      true_num += 1
      true_dif_sims.append(dif_similarity_scores[k])
      true_most_sims.append(most_similarity_scores[k])
    else: # 不正解（False Positive / False Negative）
      false_num += 1
      false_dif_sims.append(dif_similarity_scores[k])
      false_most_sims.append(most_similarity_scores[k])

  return true_num, false_num, true_dif_sims, false_dif_sims, true_most_sims, false_most_sims

## 4. Similarity-based Classification Engine
未知の入力ベクトルに対して、ベースラインとなるSafe/Outそれぞれのベクトル群とのコサイン類似度をテンソル演算によって一括計算し、最も類似度が高いクラスを予測結果とする推論エンジンです。

In [ ]:
def get_similarity(test_vectors, test_label, train_safe_vectors, train_out_vectors):
  """
  コサイン類似度に基づく分類を実行する。
  """
  # 高速計算のためテンソル化してGPU/CPUへ転送
  test_v_all = torch.tensor(np.array(test_vectors)).to(device)
  safe_v_all = torch.tensor(np.array(train_safe_vectors)).to(device)
  out_v_all = torch.tensor(np.array(train_out_vectors)).to(device)

  batch_size = 64
  source_files, dif_similarity_scores, most_similarity_scores, other_similarity_scores = [], [], [], []
  error = 0

  for i in tqdm(range(0, len(test_v_all), batch_size)):
    batch_test = test_v_all[i : i + batch_size]

    # テストデータの各バッチに対して、Safe/Out全件とのコサイン類似度を行列演算
    sim_matrix_safe = torch.nn.functional.cosine_similarity(batch_test.unsqueeze(1), safe_v_all.unsqueeze(0), dim=2)
    sim_matrix_out = torch.nn.functional.cosine_similarity(batch_test.unsqueeze(1), out_v_all.unsqueeze(0), dim=2)

    # 各クラス内での最大類似度を取得
    max_sim_safe_batch, _ = torch.max(sim_matrix_safe, dim=1)
    max_sim_out_batch, _ = torch.max(sim_matrix_out, dim=1)

    for j in range(len(batch_test)):
      ms_safe = max_sim_safe_batch[j].item()
      ms_out = max_sim_out_batch[j].item()

      # 最大類似度が高いクラスを最終的な予測として採用 (k-NN, k=1)
      if ms_safe > ms_out:
          source_files.append("safe")
          most_similarity_scores.append(ms_safe)
          other_similarity_scores.append(ms_out)
      elif ms_safe < ms_out:
          source_files.append("out")
          most_similarity_scores.append(ms_out)
          other_similarity_scores.append(ms_safe)
      else:
          source_files.append("error")
          most_similarity_scores.append(0); other_similarity_scores.append(0)
          error += 1
      # マージン（確信度）の記録: |予測クラスの最大類似度 - 他クラスの最大類似度|
      dif_similarity_scores.append(most_similarity_scores[-1] - other_similarity_scores[-1])

  return check_classification(test_label, source_files, dif_similarity_scores, most_similarity_scores)

## 5. Execution & Data Balancing
クラス不均衡によるバイアスを排除するため、ベースラインデータにアンダーサンプリング（Undersampling）を適用して均等化した後、推論・評価パイプラインを実行します。

In [ ]:
# 1. 前処理済みデータからのベクトル抽出
safe_test_vec = get_vectors("test_safe")
out_test_vec = get_vectors("test_out")
safe_train_vec = get_vectors("train_safe")
out_train_vec = get_vectors("train_out")

# 2. クラス不均衡の解消 (Undersampling)
random.seed(1) # 再現性確保のためのシード固定
if len(safe_train_vec) <= len(out_train_vec):
  out_train_vec = random.sample(out_train_vec, len(safe_train_vec))
else:
  safe_train_vec = random.sample(safe_train_vec, len(out_train_vec))

# 3. テストデータに対する推論と評価の実行
safe_true, safe_false, safe_true_dif_sims, safe_false_dif_sims, safe_true_most_sims, safe_false_most_sims = get_similarity(safe_test_vec, "safe", safe_train_vec, out_train_vec)
out_true, out_false, out_true_dif_sims, out_false_dif_sims, out_true_most_sims, out_false_most_sims = get_similarity(out_test_vec, "out", safe_train_vec, out_train_vec)

# 4. Accuracy（正解率）の算出と出力
safe_per = 100 * safe_true / (safe_true + safe_false)
out_per = 100 * out_true / (out_true + out_false)
print("\n\n")
print("=======================")
print(f"safe correct:{safe_per:.2f}%")
print(f"out correct:{out_per:.2f}%")
print(f"all percentage: {(safe_per + out_per) / 2:.2f}%")
print("=======================")
print("\n\n")

## 6. Visualization of Model Confidence
モデルの予測根拠（確信度と類似度の絶対値）を可視化し、分類結果の妥当性を定性的に評価します。

In [ ]:
# グラフ用に各予測パターン（True/False）ごとの平均スコアを計算
avg_dif = [
  np.mean(safe_true_dif_sims) if safe_true_dif_sims else 0,
  np.mean(safe_false_dif_sims) if safe_false_dif_sims else 0,
  np.mean(out_true_dif_sims) if out_true_dif_sims else 0,
  np.mean(out_false_dif_sims) if out_false_dif_sims else 0
]

avg_most = [
  np.mean(safe_true_most_sims) if safe_true_most_sims else 0,
  np.mean(safe_false_most_sims) if safe_false_most_sims else 0,
  np.mean(out_true_most_sims) if out_true_most_sims else 0,
  np.mean(out_false_most_sims) if out_false_most_sims else 0
]

labels = ['Safe_True', 'Safe_False', 'Out_True', 'Out_False']
plt.figure(figsize=(12, 5))

# 左グラフ：確信度（スコア差）の平均
plt.subplot(1, 2, 1)
plt.bar(labels, avg_dif, color='skyblue')
plt.title('Average Score Difference (Confidence)')
plt.ylabel('Score Gap')
plt.xticks(rotation=15)

# 右グラフ：最大類似度の平均
plt.subplot(1, 2, 2)
plt.bar(labels, avg_most, color='salmon')
plt.title('Average Maximum Similarity Score')
plt.ylabel('Cosine Similarity')
plt.xticks(rotation=15)

plt.tight_layout()
plt.show()